<a href="https://colab.research.google.com/github/hanadi656/QGTPrograms/blob/main/CHSH_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 61.2 MB/s eta 0:00:00


In [ ]:
#from qiskit import QuantumCircuit, Aer, execute
from qiskit import QuantumCircuit
from qiskit_aer import Aer
#from qiskit import execute
from qiskit import transpile

import numpy as np

def chsh_quantum_strategy(x, y, shots=1000):
    """Constructs and runs a CHSH quantum circuit for inputs x and y."""
    qc = QuantumCircuit(2, 2)

    # Step 1: Prepare entangled state
    qc.h(0)
    qc.cx(0, 1)
    #------------------------------------------------------------------------------
    # Step 2: Alice's measurement setting
    #if x == 0:
    #    pass  # Z basis
    #else:
    #    qc.ry(np.pi/2, 0)  # X basis

    # Step 3: Bob's measurement setting
    #if y == 0:
    #    qc.ry(-np.pi/8, 1)  # (Z+X)/√2 basis
    #else:
    #    qc.ry(np.pi/8, 1)   # (Z−X)/√2 basis
    #------------------------------------------------------------------------------------
    if x == 1:
      qc.h(0)  # X basis for Alice if x=1

    if y == 0:
      qc.ry(-np.pi/4, 1)  # -45 degrees for Bob
    else:
      qc.ry(np.pi/4, 1)   # +45 degrees for Bob
    # Step 4: Measure
    qc.measure(0, 0)
    qc.measure(1, 1)

    # Simulate
    backend = Aer.get_backend('qasm_simulator')
   #job = execute(qc, backend, shots=shots)
    job = backend.run(transpile(qc, backend), shots=shots)
    #backend = Aer.get_backend('qasm_simulator')
    result = job.result()
    counts = result.get_counts(qc)
    display(qc.draw("text"))
    return counts

# Evaluate the CHSH game win rate
total_wins = 0
total_trials = 0
shots_per_case = 1024

for x in [0, 1]:
    for y in [0, 1]:
        counts = chsh_quantum_strategy(x, y, shots=shots_per_case)
        xy = x * y  # Expected parity of a + b

        for outcome, freq in counts.items():
            a = int(outcome[1])  # Alice's bit is qubit 0 → c0
            b = int(outcome[0])  # Bob's bit is qubit 1 → c1
            if (a + b) % 2 == xy:
                total_wins += freq
            total_trials += freq

# Calculate and print win rate

win_rate = total_wins / total_trials
print(f"Total wins: {total_wins}")
print(f"Total trials: {total_trials}")

print(f"Quantum CHSH strategy win rate: {win_rate:.4f}")


┌───┐                 ┌─┐   
q_0: ┤ H ├──■──────────────┤M├───
     └───┘┌─┴─┐┌──────────┐└╥┘┌─┐
q_1: ─────┤ X ├┤ Ry(-π/4) ├─╫─┤M├
          └───┘└──────────┘ ║ └╥┘
c: 2/═══════════════════════╩══╩═
                            0  1

┌───┐                ┌─┐   
q_0: ┤ H ├──■─────────────┤M├───
     └───┘┌─┴─┐┌─────────┐└╥┘┌─┐
q_1: ─────┤ X ├┤ Ry(π/4) ├─╫─┤M├
          └───┘└─────────┘ ║ └╥┘
c: 2/══════════════════════╩══╩═
                           0  1

┌───┐        ┌───┐    ┌─┐   
q_0: ┤ H ├──■─────┤ H ├────┤M├───
     └───┘┌─┴─┐┌──┴───┴───┐└╥┘┌─┐
q_1: ─────┤ X ├┤ Ry(-π/4) ├─╫─┤M├
          └───┘└──────────┘ ║ └╥┘
c: 2/═══════════════════════╩══╩═
                            0  1

┌───┐        ┌───┐   ┌─┐   
q_0: ┤ H ├──■─────┤ H ├───┤M├───
     └───┘┌─┴─┐┌──┴───┴──┐└╥┘┌─┐
q_1: ─────┤ X ├┤ Ry(π/4) ├─╫─┤M├
          └───┘└─────────┘ ║ └╥┘
c: 2/══════════════════════╩══╩═
                           0  1

Total wins: 3488
Total trials: 4096
Quantum CHSH strategy win rate: 0.8516
